In [1]:
import pandas as pd

In [2]:
# training data
print('loading...')
train_user_info=pd.read_csv('data/train_user_info_data.csv')
train_trans=pd.read_csv('data/train_user_transaction_data.csv')
train_behavior=pd.read_csv('data/train_user_behavior_data.csv')
print('finished!')

loading...


/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished!


In [40]:
def string2value(df):
    train_data=df.set_index('id').drop(['issue_d','emp_title','zip_code','addr_state'], axis=1) #feature exaction
    train_data['grade'] = pd.factorize(train_data['grade'])[0]
    train_data['sub_grade'] = pd.factorize(train_data['sub_grade'])[0]
    train_data['emp_length'] = pd.factorize(train_data['emp_length'])[0]
    train_data['home_ownership'] = pd.factorize(train_data['home_ownership'])[0]
    train_data['verification_status'] = pd.factorize(train_data['verification_status'])[0]
    train_data['verification_status_joint'] = pd.factorize(train_data['verification_status_joint'])[0]
    if 'fraud_status' in df.columns:
        train_data.loc[train_data['fraud_status']=='Fraud_0', 'fraud_status']=1
        train_data.loc[train_data['fraud_status']=='Fraud_1', 'fraud_status']=1
        train_data.loc[train_data['fraud_status']=='Fraud_2', 'fraud_status']=1
        train_data.loc[train_data['fraud_status']=='Normal_0', 'fraud_status']=0
        train_data.loc[train_data['fraud_status']=='Normal_1', 'fraud_status']=0
        train_data.loc[train_data['fraud_status']=='Suspect_0', 'fraud_status']=0
        train_data.loc[train_data['fraud_status']=='Suspect_1', 'fraud_status']=1
        train_data.loc[train_data['fraud_status']=='Suspect_2', 'fraud_status']=1
    return train_data

In [44]:
train_data_one=string2value(train_user_info)
train_data_one.head(5)

,fraud_status,grade,sub_grade,emp_length,home_ownership,verification_status,verification_status_joint,delinq_2yrs
id,,,,,,,,
1441601,0,0,0,0,0,0,-1,0.0
1409862,1,0,1,1,1,1,-1,0.0
1794254,0,1,2,1,1,2,-1,0.0
1815442,0,0,3,2,0,0,-1,0.0
1372156,1,2,4,1,0,1,-1,0.0


In [61]:
def nan2value(df1,df2,df3):
    train_data=df2.set_index('id').join(df3.set_index('id'))
    train_data=train_data.join(df1, on='id')
    train_data.dropna(thresh=train_data.shape[0]*0.2, axis=1,inplace=True) # drop Nan columns which exceeds 20%
    train_data.fillna(value=train_data.mean(axis=0),inplace=True) # fill others with mean values each column
    return train_data

In [63]:
train_data_two=nan2value(train_data_one, train_trans, train_behavior)
train_data_two.head(5)

,trans_filed_0,trans_filed_1,trans_filed_2,trans_filed_3,trans_filed_4,trans_filed_5,trans_filed_6,trans_filed_7,mask_order_amt,trans_filed_9,...,behavior_filed_29,behavior_filed_30,fraud_status,grade,sub_grade,emp_length,home_ownership,verification_status,verification_status_joint,delinq_2yrs
id,,,,,,,,,,,,,,,,,,,,,
1441601,9.44,230.44,29.22,80000.0,0.0,14122,34.1,35.0,7.2,7200.0,...,12.0,35.77477,0,0,0,0,0,0,-1,0.0
1409862,11.99,800.62,26.80,100000.0,0.0,52838,59.9,37.0,36.0,36000.0,...,10.0,35.77477,1,0,1,1,1,1,-1,0.0
1794254,7.35,155.19,20.27,55000.0,0.0,4011,30.0,19.0,5.0,5000.0,...,0.0,33.00000,0,1,2,1,1,2,-1,0.0
1815442,9.93,161.18,10.86,60000.0,0.0,7022,59.5,8.0,5.0,5000.0,...,1.0,35.77477,0,0,3,2,0,0,-1,0.0
1372156,14.08,559.44,20.24,120000.0,0.0,17384,47.1,30.0,24.0,24000.0,...,3.0,35.77477,1,2,4,1,0,1,-1,0.0


In [84]:
def join_split(df, df1, df2):
    train_data=string2value(df)
    train_data=nan2value(train_data,df1,df2)
    if 'fraud_status' in train_data.columns: #train data
#         train_data.sample(frac=1)  
        label=train_data['fraud_status']
        data=train_data.drop(['fraud_status'],axis=1)
        return data,label
    else: #test data
        return train_data

In [85]:
train_data, train_label= join_split(train_user_info,train_trans,train_behavior)
train_data.head(5)
train_label.head(5)

id
1441601    0
1409862    1
1794254    0
1815442    0
1372156    1
Name: fraud_status, dtype: int64

In [86]:
# feature normalization
from sklearn import preprocessing
data = preprocessing.normalize(train_data, norm='l2', axis=1)
label = train_label.values
print(data, label)

[[ 3.40667475e-05  8.31603950e-04  1.05448132e-04 ...  0.00000000e+00
  -3.60876562e-06  0.00000000e+00]
 [ 2.08544205e-05  1.39253262e-03  4.66137171e-05 ...  1.73931780e-06
  -1.73931780e-06  0.00000000e+00]
 [ 2.34268935e-05  4.94642122e-04  6.46072286e-05 ...  6.37466489e-06
  -3.18733244e-06  0.00000000e+00]
 ...
 [ 5.84043769e-05  1.24779944e-03  1.12136404e-04 ...  8.05577613e-06
  -4.02788807e-06  0.00000000e+00]
 [ 7.08654131e-05  2.34240024e-03  6.71872774e-05 ...  8.17363473e-06
  -4.08681736e-06  0.00000000e+00]
 [ 6.94618352e-05  1.52391843e-03  9.47095604e-05 ...  8.15758487e-06
  -4.07879244e-06  0.00000000e+00]] [0 1 0 ... 0 0 0]


In [89]:
threshold=int(len(data)*0.8)
train=data[:threshold]
valid=data[threshold:-1]
train_label=label[:threshold]
valid_label=label[threshold:-1]
print(train,train_label)
print(valid,valid_label)

[[ 3.40667475e-05  8.31603950e-04  1.05448132e-04 ...  0.00000000e+00
  -3.60876562e-06  0.00000000e+00]
 [ 2.08544205e-05  1.39253262e-03  4.66137171e-05 ...  1.73931780e-06
  -1.73931780e-06  0.00000000e+00]
 [ 2.34268935e-05  4.94642122e-04  6.46072286e-05 ...  6.37466489e-06
  -3.18733244e-06  0.00000000e+00]
 ...
 [ 6.26601528e-05  2.43097446e-03  8.96855508e-05 ...  5.70156077e-06
  -5.70156077e-06  0.00000000e+00]
 [ 1.01800207e-04  5.24180863e-03  1.20871638e-04 ...  1.28861021e-05
  -1.28861021e-05  0.00000000e+00]
 [ 1.76451077e-04  2.62366100e-03  3.24891997e-04 ...  2.58158123e-05
  -1.29079061e-05  0.00000000e+00]] [0 1 0 ... 0 0 0]
[[ 4.19275003e-05  1.49102312e-03  4.91278874e-05 ...  0.00000000e+00
  -2.36078267e-06  0.00000000e+00]
 [ 2.63914819e-06  2.45397114e-04  2.39351909e-06 ...  2.72921219e-07
  -2.72921219e-07  0.00000000e+00]
 [ 3.85983099e-04  7.36281253e-03  0.00000000e+00 ...  0.00000000e+00
  -1.50951544e-05  0.00000000e+00]
 ...
 [ 4.94506006e-05  6.85781

In [90]:
# xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(train, label=train_label)
dvalid=xgb.DMatrix(valid, label=valid_label)

In [176]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic','eval_metric':'error'}
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
num_round = 15
bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=10)
bst.save_model('xgb.model')

[0]	eval-error:0.154452	train-error:0.155269
Multiple eval metrics have been passed: 'train-error' will be used for early stopping.

Will train until train-error hasn't improved in 10 rounds.
[1]	eval-error:0.105506	train-error:0.103616
[2]	eval-error:0.094567	train-error:0.092939
[3]	eval-error:0.076432	train-error:0.074586
[4]	eval-error:0.067451	train-error:0.065351
[5]	eval-error:0.066468	train-error:0.064305
[6]	eval-error:0.06256	train-error:0.060199
[7]	eval-error:0.061682	train-error:0.059238
[8]	eval-error:0.058096	train-error:0.055166
[9]	eval-error:0.05864	train-error:0.055966
[10]	eval-error:0.064429	train-error:0.061781
[11]	eval-error:0.057838	train-error:0.054747
[12]	eval-error:0.065819	train-error:0.062594
[13]	eval-error:0.056766	train-error:0.053543
[14]	eval-error:0.054921	train-error:0.051535


In [103]:
# testing data
print('loading...')
test_user_info=pd.read_csv('data/test_user_info_data.csv')
test_trans=pd.read_csv('data/test_user_transaction_data.csv')
test_behavior=pd.read_csv('data/test_user_behavior_data.csv')
print('finished!')

loading...
finished!


In [104]:
test_data= join_split(test_user_info,test_trans,test_behavior)
test_data.head(5)

,trans_filed_0,trans_filed_1,trans_filed_2,trans_filed_3,trans_filed_4,trans_filed_5,trans_filed_6,trans_filed_7,mask_order_amt,trans_filed_9,...,behavior_filed_28,behavior_filed_29,behavior_filed_30,grade,sub_grade,emp_length,home_ownership,verification_status,verification_status_joint,delinq_2yrs
id,,,,,,,,,,,,,,,,,,,,,
1398633,10.47,162.45,14.11,25000.0,1.0,1840,11.7,13.0,5.0,5000.0,...,39.84335,5.0,36.098464,0,0,-1,0,0,-1,0.0
208570,7.21,650.44,3.92,176000.0,0.0,9663,4.0,47.0,21.0,21000.0,...,39.84335,7.0,36.098464,1,1,0,1,1,-1,0.0
1552832,6.67,460.90,17.43,73000.0,0.0,9249,25.9,29.0,15.0,15000.0,...,25.00000,6.0,25.000000,1,2,0,1,2,-1,0.0
1190282,14.47,275.26,15.06,63500.0,1.0,14810,35.9,32.0,8.0,8000.0,...,39.84335,5.0,36.098464,2,3,0,0,2,-1,0.0
1469377,17.97,216.83,17.56,65000.0,0.0,2263,29.4,11.0,6.0,6000.0,...,29.00000,0.0,29.000000,3,4,1,0,2,-1,0.0


In [105]:
data = preprocessing.normalize(test_data, norm='l2', axis=1)

In [155]:
dtest = xgb.DMatrix(data)
ypred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
print(ypred)
print(len(ypred))

[0.2471767  0.04628647 0.04628647 ... 0.1350425  0.14559165 0.03155841]
95888


In [173]:
ypred[ypred<0.5]=0
ypred[ypred>0.5]=1
print(ypred)
print(len(ypred[ypred==0]),len(ypred[ypred==1]))
ypred=ypred.astype(np.int64)
print(ypred.dtype)

SyntaxError: can't assign to function call (<ipython-input-173-00f05560ccf9>, line 1)

In [171]:
index=np.arange(len(ypred))
print(index.dtype)
print(index.reshape(-1,1))
print(ypred.reshape(-1,1))
np_data=np.hstack((index.reshape(-1,1), ypred.reshape(-1,1)))
print(np_data)

int64
[[    0]
 [    1]
 [    2]
 ...
 [95885]
 [95886]
 [95887]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[    0     0]
 [    1     0]
 [    2     0]
 ...
 [95885     0]
 [95886     0]
 [95887     0]]


In [172]:
pd_data = pd.DataFrame(np_data,columns=['ID','result'])
pd_data.to_csv('result.csv')